# High-level RNN PyTorch Example

*Modified by Jordan A Caraballo Vega (jordancaraballo)*

In [1]:
import os
import sys
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import torch.nn.init as init
from torch import autograd
from torch.autograd import Variable
from common.params_lstm import *
from common.utils import *

In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("PyTorch: ", torch.__version__)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.6.7 | packaged by conda-forge | (default, Jul  2 2019, 02:18:42) 
[GCC 7.3.0]
PyTorch:  1.4.0
Numpy:  1.17.0
GPU:  ['Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB']
CUDA Version 10.1.243
CuDNN Version  No CUDNN in this machine


In [4]:
class SymbolModule(nn.Module):
    def __init__(self, 
                 maxf=MAXFEATURES, edim=EMBEDSIZE, nhid=NUMHIDDEN):
        super(SymbolModule, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=maxf,
                                      embedding_dim=edim)
        # If batch-first then input and output 
        # provided as (batch, seq, features)
        # Cudnn used by default if possible
        self.gru = nn.GRU(input_size=edim, 
                          hidden_size=nhid, 
                          num_layers=1,
                          batch_first=True,
                          bidirectional=False)   
        self.l_out = nn.Linear(in_features=nhid*1,
                               out_features=2)

    def forward(self, x, nhid=NUMHIDDEN, batchs=BATCHSIZE):
        x = self.embedding(x)
        h0 = Variable(torch.zeros(1, batchs, nhid)).cuda()
        x, h = self.gru(x, h0)  # outputs, states
        # just get the last output state
        x = x[:,-1,:].squeeze()
        x = self.l_out(x)
        return x

In [5]:
def init_model(m, lr=LR, b1=BETA_1, b2=BETA_2, eps=EPS):
    opt = optim.Adam(m.parameters(), lr, betas=(b1, b2), eps=eps)
    criterion = nn.CrossEntropyLoss()
    return opt, criterion

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES)
# Torch-specific
x_train = x_train.astype(np.int64)
x_test = x_test.astype(np.int64)
y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
Trimming to 30000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000,) (25000,)
int64 int64 int64 int64
CPU times: user 4.94 s, sys: 343 ms, total: 5.29 s
Wall time: 5.65 s


In [7]:
%%time
sym = SymbolModule()
sym.cuda() # CUDA!

CPU times: user 2.43 s, sys: 1.1 s, total: 3.53 s
Wall time: 3.55 s


In [8]:
%%time
optimizer, criterion = init_model(sym)

CPU times: user 131 µs, sys: 87 µs, total: 218 µs
Wall time: 224 µs


In [9]:
%%time
# Main training loop: 12.7s
sym.train() # Sets training = True  
EPOCHS=40
for j in range(EPOCHS):
    for data, target in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        # Get samples
        data = Variable(torch.LongTensor(data).cuda())
        target = Variable(torch.LongTensor(target).cuda())
        # Init
        optimizer.zero_grad()
        # Forwards
        output = sym(data)
        # Loss
        loss = criterion(output, target)
        # Back-prop
        loss.backward()
        optimizer.step()
    # Log
    print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
CPU times: user 3min 16s, sys: 3.11 s, total: 3min 19s
Wall time: 3min 19s


In [10]:
%%time
# Main evaluation loop: 1.52s
sym.eval() # Sets training = False
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, target in yield_mb(x_test, y_test, BATCHSIZE):
    # Get samples
    data = Variable(torch.LongTensor(data).cuda())
    # Forwards
    output = sym(data)
    pred = output.data.max(1)[1].cpu().numpy().squeeze()
    # Collect results
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = pred
    c += 1

CPU times: user 1.2 s, sys: 47 ms, total: 1.25 s
Wall time: 1.25 s


In [11]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.8561698717948718
